In [1]:
#Example Notebook
from datetime import datetime
from refit.api.refit import Refit

In [2]:
project_guid = "b6ee5bab-08dd-49b0-98b6-45cd0a28b12f"
refit = Refit(project_guid)

start = datetime(2020, 6, 20)
end = datetime(2020, 6, 30)
sensors = ['5163']

/Users/matthewalvarez/Documents/projects/iot-prototype/training/refit/util/Schema.py:23: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict = yaml.load(schema_yaml)


In [3]:
# Getting Sensor Data with the training window flag incorporated
df = refit.sensor_data_with_flag(start, end, sensors)
df

,project_guid_x,sensor_id,partition_key,timestamp,temperature,pressure,wind,operable
0,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,True
1,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,True
2,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,True
3,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,True
4,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:42,68.0,1028.0,64.0,True
5,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:42,68.0,1028.0,64.0,True
6,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:47,41.0,1137.0,2.0,True
7,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:47,41.0,1137.0,2.0,True
8,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:52,41.0,1014.0,53.0,True
9,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:52,41.0,1014.0,53.0,True


In [4]:
# Getting Sensor Data alone
df = refit.sensor_data_with_flag(start, end, sensors)
df

,project_guid_x,sensor_id,partition_key,timestamp,temperature,pressure,wind,operable
0,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,True
1,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,True
2,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,True
3,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,True
4,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:42,68.0,1028.0,64.0,True
5,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:42,68.0,1028.0,64.0,True
6,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:47,41.0,1137.0,2.0,True
7,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:47,41.0,1137.0,2.0,True
8,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:52,41.0,1014.0,53.0,True
9,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:48:52,41.0,1014.0,53.0,True


In [7]:
# Getting training windows alone
df = refit.get_training_windows(start, end, sensors)
df

,project_guid,sensor_id,partition_key,start,end
0,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 00:50:00,2020-06-28 02:00:00
1,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 00,2020-06-28 05:50:00,2020-06-28 07:00:00
0,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 10,2020-06-28 10:00:00,2020-06-28 12:10:00
1,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 10,2020-06-28 15:00:00,2020-06-28 17:10:00
0,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 13,2020-06-28 13:30:00,2020-06-28 15:15:00
1,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 13,2020-06-28 18:30:00,2020-06-28 20:15:00
0,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 15,2020-06-28 15:45:00,2020-06-28 16:50:00
1,b6ee5bab-08dd-49b0-98b6-45cd0a28b12f,5163,2020-06-28 15,2020-06-28 20:45:00,2020-06-28 21:50:00
